
# Import Python libraries.

In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer,SimpleImputer,IterativeImputer
from sklearn.preprocessing import LabelEncoder,normalize,LabelBinarizer,OneHotEncoder
from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split ,GridSearchCV ,cross_val_predict,cross_val_score
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier

# Import Datasets

In [77]:
train_data = pd.read_csv(r"Training Data.csv")
test_data = pd.read_csv(r"Test Data.csv")
train_data


,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0
251996,251997,2843572,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251997,251998,4522448,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,251999,6507128,45,0,single,rented,no,Graphic_Designer,Pondicherry,Puducherry,0,10,0


In [78]:
test_data

,id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,1,7393090,59,19,single,rented,no,Geologist,Malda,West Bengal,4,13
1,2,1215004,25,5,single,rented,no,Firefighter,Jalna,Maharashtra,5,10
2,3,8901342,50,12,single,rented,no,Lawyer,Thane,Maharashtra,9,14
3,4,1944421,49,9,married,rented,yes,Analyst,Latur,Maharashtra,3,12
4,5,13429,25,18,single,rented,yes,Comedian,Berhampore,West Bengal,13,11
...,...,...,...,...,...,...,...,...,...,...,...,...
27995,27996,9955481,57,13,single,rented,no,Statistician,Eluru[25],Andhra Pradesh,5,10
27996,27997,2917765,47,9,single,rented,no,Technical writer,Ratlam,Madhya Pradesh,9,14
27997,27998,8082415,24,5,single,rented,no,Lawyer,Mira-Bhayandar,Maharashtra,4,13
27998,27999,9474180,51,13,single,rented,yes,Chartered Accountant,Bhilai,Chhattisgarh,13,14


# Check how train dataset looks like.

In [79]:
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


# Convert datatype of selected fields.

In [80]:
train_data["profession"]=pd.factorize(train_data.profession)[0]
train_data["city"]=pd.factorize(train_data.city)[0]
train_data["state"]=pd.factorize(train_data.state)[0]
train_data["married"]=pd.factorize(train_data.married)[0]
train_data["house_ownership"]=pd.factorize(train_data.house_ownership)[0]
train_data["car_ownership"]=pd.factorize(train_data.car_ownership)[0]

In [81]:
test_data["profession"]=pd.factorize(test_data.profession)[0]
test_data["city"]=pd.factorize(test_data.city)[0]
test_data["state"]=pd.factorize(test_data.state)[0]
test_data["married"]=pd.factorize(test_data.married)[0]
test_data["house_ownership"]=pd.factorize(test_data.house_ownership)[0]
test_data["car_ownership"]=pd.factorize(test_data.car_ownership)[0]


In [82]:
print(test_data)
print("training data")
print(train_data)

          id   income  age  experience  married  house_ownership  \
0          1  7393090   59          19        0                0   
1          2  1215004   25           5        0                0   
2          3  8901342   50          12        0                0   
3          4  1944421   49           9        1                0   
4          5    13429   25          18        0                0   
...      ...      ...  ...         ...      ...              ...   
27995  27996  9955481   57          13        0                0   
27996  27997  2917765   47           9        0                0   
27997  27998  8082415   24           5        0                0   
27998  27999  9474180   51          13        0                0   
27999  28000  9250350   42           9        0                0   

       car_ownership  profession  city  state  current_job_years  \
0                  0           0     0      0                  4   
1                  0           1     1      1  

# Drop the dependent variable from the train/test dataset. 

In [85]:
X_train=train_data.drop("risk_flag",axis=1)
y_train=train_data["risk_flag"]
X_test=test_data.drop("risk_flag",axis=1)
y_test=test_data["risk_flag"]
print(train_data)

KeyError: 'risk_flag'

# Drop unnamed field from train and test dataset.

In [ ]:
# x_train_Unnamed  = X_train.pop("Unnamed: 0")
# x_test_Unnamed  = X_test.pop("Unnamed: 0")


In [ ]:
X_train.head()

In [135]:
def print_score(clf, X_train, X_test, y_train, y_test,c,i):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    #labels = ['True Neg','False Pos','False Neg','True Pos']
    #labels = np.asarray(labels).reshape(2,2)
    lb = LabelBinarizer()
    lb.fit(y_train)
    print("\ntraining performance\n")
    '''
    training performance
    '''
    res = clf.predict(X_train)
    print("Train Result:\n")
    print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train,res)))
    print("Classification Report: \n {}\n".format(classification_report(y_train,res)))
    print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train,res)))
    plt.figure(i)
    plt.title("{} training".format(c), fontsize =20)
    sns.heatmap(confusion_matrix(y_train,res), annot=True,cmap="YlGnBu")
    print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), lb.transform(res))))
        
    print("\ntest performance\n")
    '''
    test performance
    '''
    res_test = clf.predict(X_test)
    print("Test Result:\n")        
    print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test,res_test)))
    print("Classification Report: \n {}\n".format(classification_report(y_test,res_test)))
    print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test,res_test))) 
    plt.figure(i+1)
    plt.title("{} test".format(c), fontsize =20)
    sns.heatmap(confusion_matrix(y_test, res_test), annot=True,cmap="YlGnBu")
    print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test),lb.transform(res_test))))

# Train your model

In [86]:
from sklearn.tree import DecisionTreeClassifier
DTClassifier= DecisionTreeClassifier(criterion='entropy', random_state=0)
DTClassifier.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [136]:
clf={}
clf["svc"]=SVC(kernel='poly')
clf["DT"]=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=5, min_samples_leaf=5) 
clf["knn"]=KNeighborsClassifier(n_neighbors=5,p=2, metric='minkowski')
clf["RF"]=RandomForestClassifier(n_estimators=200,random_state=100,max_depth=5,max_features='auto')
clf["AD"]=AdaBoostClassifier(n_estimators=200,random_state = 100)
clf["GB"]=GradientBoostingClassifier(n_estimators=200,random_state = 100,max_features="auto")
clf["xgb"]=xgb.XGBClassifier(learning_rate=0.1,n_estimators=200,objective='binary:logistic',nthread= 4,n_jobs= 4,base_score= 0.6)
clf["AD_RF"]=AdaBoostClassifier(base_estimator=clf["RF"],n_estimators=200,random_state = 100)
clf["BAGG"]=BaggingClassifier(base_estimator=clf["DT"], n_estimators=1000, bootstrap=True, oob_score=True,n_jobs=-1, random_state=100)
clf["St"]=StackingClassifier(estimators=[('DT',clf["DT"] ),('RFC', clf["RF"])],final_estimator=LogisticRegression())

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
j=0
for i in clf:
    print("c:",i)
    clf[i].fit(X_train,y_train)
    print_score(clf[i], X_train, X_test, y_train, y_test,i,j)
    j+=2
  

(252000, 12)
(28000, 12)
(252000,)
(251999,)
c: svc


# Test your model 

In [87]:
y_pred=DTClassifier.predict(X_test)

In [88]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [107]:
df1=pd.DataFrame(y_pred)
df2=pd.DataFrame(test_data["id"])
df2
result = pd.concat([df2, df1], axis=1)

print(result.shape)
result.head(10)

(28000, 2)


,id,0
0,1,0
1,2,0
2,3,0
3,4,0
4,5,1
5,6,0
6,7,0
7,8,0
8,9,0
9,10,1


,id
0,1
1,2
2,3
3,4
4,5
...,...
27995,27996
27996,27997
27997,27998
27998,27999


In [130]:
yop=y_test.sample(28000)
yop

224277    0
16072     0
69534     0
195703    1
150742    0
         ..
145981    0
214062    0
56491     0
67442     1
75274     0
Name: risk_flag, Length: 28000, dtype: int64

# Check your roc_auc_score 

In [131]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_pred, yop)
print('ROC AUC: %f' % auc)


ROC AUC: 0.496743


In [108]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        res = clf.predict(X_train)
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, 
                                                                res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, 
                                                                            res)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, 
                                                                  res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                      lb.transform(res))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, 
                                                                res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, 
                                                                            res_test)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, 
                                                                  res_test)))   
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), 
                                                      lb.transform(res_test))))

In [133]:
print_score(DTClassifier, X_train, X_test, y_train, yop, train=False)

Test Result:

accuracy score: 0.7612

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.85      0.86     24555
           1       0.12      0.14      0.13      3445

    accuracy                           0.76     28000
   macro avg       0.50      0.50      0.50     28000
weighted avg       0.78      0.76      0.77     28000


Confusion Matrix: 
 [[20815  3740]
 [ 2947   498]]

ROC AUC: 0.4961

